# Overview

This notebook provides instructions for using the Terrain API to automate some tasks that require a little more work in the CyVerse Discovery Environment (DE).

## Target Audience

- Users interested in automating their tasks.
- Users who want to integrate CyVerse resources into their applications.
- Users looking for an introduction to using APIs in order to make use of other APIs related to CyVerse.

## Why Use Terrain?

The DE itself provides a lot of tools for managing and analyzing data, so one question that may come up is: why bother to use Terrain directly when I can use the DE itself? This is absolutely a valid argument, but there are some situations where a graphical user interface can become a bit of a hinderance. Suppose, for example, that you need to launch dozens of analyses that all use the same app with slightly different parameter values. The DE currently provides no way to do this unless the parameters being varied happen to refer to input files. Launching so many similar jobs using a GUI would be tedious and error prone. On the other hand, if you're making calls to Terrain directly, you can write a short script to quickly launch all of the analyses with all of the required parameter variations.

Of course, there is a trade-off. Making calls directly to the API does take some effort; it means that you have to perform all of these tasks:

1. Authenticate to terrain.
1. Identify the app that you want to run.
1. Obtain information about the app parameters.
1. Launch the app.
1. Send a notification when the job is done.

The DE manages all of these tasks for you. If the DE suits your needs, by all means, use it. If the tasks that you have to perform become repetitive, however, investing a little time in writing a script to automate job submission might actually save you some time in the long run.

## Documentation

Terrain has two primary sources of documentation:

- Latest and Greatest: https://de.cyverse.org/terrain/docs
- Older Documentation: https://cyverse-de.github.io/api

In most cases, the latest documentation is the documentation that you'll want to use because some of the older documentation is out of date. The only time that the older documentation is preferable is when the newer documentation hasn't been written for an endpoint that you want to use. If you use the older documentation, the best place to look is the (endpoint index)[https://cyverse-de.github.io/api/endpoint-index.html]. This page includes a list of links to all of the older DE documentation.

# Prerequisites

Before actually calling terrain, we'll have to make sure that we have all of the libraries that we need, and that we have credentials that we can use to let Terrain know who we are.

## Libraries

We'll be making extensive use of the (Requests library)[https://requests.readthedocs.io/en/master/] makes calling APIs quite simple. We'll also need to be able to prompt for a password and pretty-print some data structures.

In [ ]:
import getpass
import pprint
import requests

## Authenticating

Terrain uses OAuth2 for most endpoints. This works well for the DE because it's already integrated with an identity provider that is capable of providing tokens. For direct API calls, however, we needed something a little more convenient. For this purpose, we created a set of endpoints dedicated to obtaining OAuth2 tokens. These endpoints use HTTP basic (that is, username and password) authentication and, assuming the credentials are valid, return a token that can be used to call other Terrain endpoints. In Python, supporting HTTP basic authentication means that we have to prompt for a username and password. This is where the `getpass` library comes in.

In [ ]:
print("Username: ", end='', flush=True)
username = input()
print("Password: ", end='', flush=True)
password = getpass.getpass()

Now that we have the username and password, we can obtain the authentication token by calling the `/terrain/token` endpoint.

In [ ]:
r = requests.get("https://de.cyverse.org/terrain/token", auth=(username, password))
r.raise_for_status()
token = r.json()['access_token']
auth_headers = {"Authorization": "Bearer " + token}

# Example 1: VICE Analysis

Launching VICE analyses in the DE is actually quite simple, but the simplicity of this task makes it an ideal first example.

## Finding the App

The first step is to find a VICE app to use. For this task, I'm going to use the app, `JupyterLab-with-sql-1.0.9`. The first step is to get the app ID so that we can get information about how to launch the analysis. For this step, we're going to use the app search endpoint.

In [ ]:
query_params = {"search": "JupyterLab-with-sql-1.0.9"}
r = requests.get("https://de.cyverse.org/terrain/apps", headers=auth_headers, params=query_params)
r.raise_for_status()
pprint.pprint(r.json())

## Obtaining the App Details

Armed with some information about the app, we can now obtain the full app description, which contains all of the information necessary to launch an analysis using the app. The first step in doing that is to obtain he information that we need from the app search above. For this step, we need the system ID and the app ID. The system ID refers to the system that is responsible for managing the app. Currently there are two valid system IDs: `de` and `agave`. Apps that use the system ID, `de`, are defined in and managed by the DE itself. Apps that use the system ID, `agave`, are defined in and managed by the (Tapis API)[https://tapis-project.org], formerly known as Agave. Of course, the app ID refers to the app itself.

In [ ]:
app_listing = r.json()["apps"][0]
system_id = app_listing["system_id"]
app_id = app_listing["id"]
print("System ID: ", system_id)
print("App ID: ", app_id)

Now we can obtain the full app description.

In [ ]:
url = "https://de.cyverse.org/terrain/apps/{0}/{1}".format(system_id, app_id)
r = requests.get(url, headers=auth_headers)
r.raise_for_status()
pprint.pprint(r.json())

## Building the Analysis Submission

This is by far the most complicated step of submitting a job using Terrain.